# フロー
1. 変数の定義
1. 過去のCSVファイルからデータをロードする
1. 前処理モジュールの定義（当日の上昇率、過去からの上昇率、下降率）
1. CSVロードして、並び替え、必要な日付からにする
1. 前処理部分(現在の値と、過去のからの上昇率、下降率を記したDFを作成)
1. 前処理データの保存

In [13]:
# 変数の定義
import pandas as pd
import pandas_profiling as pdp
import datetime
import pickle

class Make_data():
    
    pram_predict_day=3                   # 1日後の予測をする
    pd.set_option('display.max_columns', 100)
    
    def load(self):
        df = pd.read_pickle('data/stock.pkl')
        #print ("配列長さ:",len(df))
        return df
    
    
    
    # 前処理モジュールの定義（当日の上昇率、過去からの上昇率、下降率）
    # 偏差値
    def Deviation_value(self,df_tmp,x):
        std=df_tmp.std()['Open']
        mean=df_tmp.mean()['Open']
        deviation=(mean-x)*10/std+50
        return deviation
    # 前処理モジュールの定義（当日の上昇率、過去からの上昇率、下降率）

    def persent(self,day0,xday):         #　当日のopen価格と、過去のと比較して上昇率(当日open/過去open)を返す
        rate=0  #上昇率
        if ((i-xday)>=0 and data_len>(i-xday)):        
            rate= ((df_fx.iloc[i,open_index]/df_fx.iloc[i-xday,open_index])-1)*100
        return rate

    def result(self,day0,xday):          #day0 当日の上昇率(close/open)の結果を返す

        rate=0  #上昇率
        if ((i-xday)>=0 and data_len>(i-xday)): 
            rate= ((df_fx.iloc[i,close_index]/df_fx.iloc[i,open_index])-1)*100
        return rate
    
    
    def main(self,start_date):
        if start_date=="":
            start_date='2000-01-01'             #元データを2000年からにする
        else:
            self.start_date=start_date
        
        df=self.load()

        df      =df[(df['Date']>=start_date )]
        df      = df.reset_index(drop=True)
        df_len  =len(df)


        # Index の取得
        index_date    = df.columns.get_loc('Date')
        index_close   = df.columns.get_loc('Close')
        index_open    = df.columns.get_loc('Open')
        index_dow     = df.columns.get_loc('dow_compare')
        
        
        ## 日毎の９０日前からの下落率を９０日間を計算
        columns=["day+"+str(x+1) for x in range(90)]
        #print(columns)
        for i,x in enumerate(columns):
            # ＋は、前日からの比較
            df[x]=df['Open'].pct_change(periods=(i+1))*100

        ## 偏差値を計算
        # 前処理部分　（現在の値と、過去のからの上昇率、下降率を記したDFを作成
        for i in range(df_len) :
            day0_open=df.iloc[i,index_open]
            if i>90:
                df_tail=df.iloc[i-90:i,:]
                Deviation30=self.Deviation_value(df_tail.tail(30),day0_open)
                Deviation60=self.Deviation_value(df_tail.tail(60),day0_open)
                Deviation90=self.Deviation_value(df_tail.tail(90),day0_open)
            if i%100==0:
                print("進捗: {:.1%}".format(i/df_len),end=" ")
                
        #正解      # -は、未来との比較
        # １日先のオープンの値を予想
        #df['result']=df['Open'].pct_change(periods=-1*self.pram_predict_day)*100*(-1)
        # １日先のオープンの値を予想
        df['result_1day']=df['Close'].pct_change(periods=-1*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_1day']=df['result_1day'].apply(lambda x: 1 if x<0 else 0)

        df['result_3day']=df['Close'].pct_change(periods=-3*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_3day']=df['result_3day'].apply(lambda x: 1 if x<0 else 0)

        df['result_5day']=df['Close'].pct_change(periods=-5*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_5day']=df['result_5day'].apply(lambda x: 1 if x<0 else 0)
        
        
        df['result_10day']=df['Close'].pct_change(periods=-10*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_10day']=df['result_10day'].apply(lambda x: 1 if x<0 else 0)

        
        df['result_20day']=df['Close'].pct_change(periods=-20*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_20day']=df['result_20day'].apply(lambda x: 1 if x<0 else 0)

        df['result_30day']=df['Close'].pct_change(periods=-30*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_30day']=df['result_30day'].apply(lambda x: 1 if x<0 else 0)

 

        ##########################
        df['result_1day_open']=df['Open'].pct_change(periods=-1*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_1day_open']=df['result_1day_open'].apply(lambda x: 1 if x<0 else 0)

        df['result_3day_open']=df['Open'].pct_change(periods=-3*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_3day_open']=df['result_3day_open'].apply(lambda x: 1 if x<0 else 0)

        df['result_5day_open']=df['Open'].pct_change(periods=-5*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_5day_open']=df['result_5day_open'].apply(lambda x: 1 if x<0 else 0)
        
        
        df['result_10day_open']=df['Open'].pct_change(periods=-10*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_10day_open']=df['result_10day_open'].apply(lambda x: 1 if x<0 else 0)

        
        df['result_20day_open']=df['Open'].pct_change(periods=-20*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_20day_open']=df['result_20day_open'].apply(lambda x: 1 if x<0 else 0)

        df['result_30day_open']=df['Open'].pct_change(periods=-30*self.pram_predict_day)*100*(-1)
        # 上昇(買い)はマイナス１、下降（売り）は、0
        df['result_30day_open']=df['result_30day_open'].apply(lambda x: 1 if x<0 else 0)

        # １日のOpenから Closeの変化だけを見るのであればこの記述にする
        #df['result']=df['Close']-df["Open"]#/.apply(lambda x: 1 if x<0 else 0)
        #df['result']=df['result'].apply(lambda x: 1 if x>0 else 0)
        
        df=df.dropna()

        # 前処理データの保存
        if start_date<='2000-01-01': 
            df.to_pickle('data/stock_preprocessing_20201017.pkl',)
            print("【完了】データ保存")


In [14]:
df_test = pd.DataFrame({'price':  range(120, 110,1)})
pram_predict_day=1                   # 1日後の予測をする
df_test['result']=df_test['price'].pct_change(periods=-1*pram_predict_day)*100*(-1)#デフォルトは1で1行前
df_test.tail(3)

,price,result


In [15]:
%%time
start_date="2020-09-01"
m=Make_data()
m.main("")

進捗: 0.0% 進捗: 2.0% 進捗: 4.0% 進捗: 6.1% 進捗: 8.1% 進捗: 10.1% 進捗: 12.1% 進捗: 14.2% 進捗: 16.2% 進捗: 18.2% 進捗: 20.2% 進捗: 22.2% 進捗: 24.3% 進捗: 26.3% 進捗: 28.3% 進捗: 30.3% 進捗: 32.3% 進捗: 34.4% 進捗: 36.4% 進捗: 38.4% 進捗: 40.4% 進捗: 42.5% 進捗: 44.5% 進捗: 46.5% 進捗: 48.5% 進捗: 50.5% 進捗: 52.6% 進捗: 54.6% 進捗: 56.6% 進捗: 58.6% 進捗: 60.7% 進捗: 62.7% 進捗: 64.7% 進捗: 66.7% 進捗: 68.7% 進捗: 70.8% 進捗: 72.8% 進捗: 74.8% 進捗: 76.8% 進捗: 78.9% 進捗: 80.9% 進捗: 82.9% 進捗: 84.9% 進捗: 86.9% 進捗: 89.0% 進捗: 91.0% 進捗: 93.0% 進捗: 95.0% 進捗: 97.0% 進捗: 99.1% 【完了】データ保存
CPU times: user 8min 15s, sys: 3.91 s, total: 8min 19s
Wall time: 9min 1s
